In [1]:
# default_exp ipython_magic

# node magic

In [2]:
#export

import sys
sys.path.append('..')
import lib_py.util as util
from IPython.core.magic import register_cell_magic

_node_cache = ''
_notebook_name = None

@register_cell_magic
def node(arg, cell, test=False):
    global _node_cache
    global _notebook_name

    ipython_node_path = '_ipython_node.ts'
    
    def split_imports(code): return util.list_partition(code.splitlines(), lambda line: line.startswith('import '))
    cache_imports, cache_exports = split_imports(_node_cache)
    cell_imports, cell_exports = split_imports(cell)
    cell_exports = '\n'.join(cell_exports).strip('\n').split('\n')

    if arg == 'run':
        ts_imports = cache_imports
        ipython_imports = cache_imports + cell_imports

        ts_exports = cache_exports
        ipython_exports = cache_exports \
            + ([] if cell_exports == [''] 
            else (['', '// cell', 'var run = async (arg: string) => {'] + cell_exports + ['}', f'await run("{arg}")']))
    elif arg == 'export':
        ts_imports = cache_imports + cell_imports
        ipython_imports = ts_imports

        ts_exports = cache_exports + ([] if cell_exports == [''] else (['', '// cell'] + cell_exports))
        ipython_exports = ts_exports

    def join_imports(imports, exports): 
        return ''.join([
            '\n'.join(list(dict.fromkeys(imports))),
            '' if len(imports) == 0 or len(exports) == 0 else '\n\n\n',
            '\n'.join(exports).strip('\n') + '\n'
        ])
    new_code_ts = join_imports(ts_imports, ts_exports)
    new_code_ipython = join_imports(ipython_imports, ipython_exports)

    def cwpath(arg): return os.path.abspath(os.path.join(os.getcwd(), '..', 'lib_ts', arg))

    util.write_file(cwpath(ipython_node_path), new_code_ipython)

    result = util.run_node(cwpath(ipython_node_path)).splitlines() if arg == 'run' else []

    _node_cache = new_code_ts

    if test:
        return result, new_code_ts, new_code_ipython
    else:
        if new_code_ts != '\n':
            if _notebook_name is None:
                _notebook_name = util.get_notebook_name()

            ts_node_path = cwpath(f'{_notebook_name}.ts')

            util.write_file(ts_node_path, new_code_ts)
            
            return result if arg == 'run' else ts_node_path
        
        return result


# node test

In [3]:
_node_cache = ''

def test_cell_node(arg, cell, expected_ts, expected_ipython, expected_result):
    result, new_code_ts, new_code_ipython = node(arg, cell, test=True)
    if new_code_ts != expected_ts or new_code_ipython != expected_ipython or result != expected_result:
        raise Exception('\n' + '\n'.join(['\n|||arg:', arg, '\n|||cell:', cell, 
                        '\n|||new_code_ts:', new_code_ts, '\n|||expected_ts:', expected_ts, 
                        '\n|||new_code_ipython:', new_code_ipython, '\n|||expected_ipython:', expected_ipython, 
                        '\n|||result:', str(result), '\n|||expected_result:', str(expected_result)]))
    else:
        print('arg:', arg, 'result:', expected_result)

test_cell_node(
    'export',
    'import * as os from "os"',
    'import * as os from "os"\n',
    'import * as os from "os"\n',
    [])

test_cell_node(
    'export',
    'import * as os from "os"\nvar a = os.EOL',
    'import * as os from "os"\n\n\n// cell\nvar a = os.EOL\n',
    'import * as os from "os"\n\n\n// cell\nvar a = os.EOL\n',
    [])

test_cell_node(
    'export',
    'import * as fs from "fs"\nvar b = fs.constants.F_OK',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    [])

test_cell_node(
    'run',
    'console.log(a)',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar run = async (arg: string) => {\nconsole.log(a)\n}\nawait run("run")\n',
    [])

test_cell_node(
    'run',
    'console.log(b)',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar run = async (arg: string) => {\nconsole.log(b)\n}\nawait run("run")\n',
    ['0'])

test_cell_node(
    'export',
    'import * as fs from "fs"\nvar c = fs.constants.X_OK',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar c = fs.constants.X_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar c = fs.constants.X_OK\n',
    [])

_node_cache = ''

arg: export result: []
arg: export result: []
arg: export result: []
arg: run result: []
arg: run result: ['0']
arg: export result: []


# spiral magic

In [4]:
#export

import shutil
import os
import time
from IPython.core.magic import register_cell_magic

def _get_spiral_cache_empty(): return {'spi': '', 'spir': '', 'run': '', '': ''}
_spiral_cache = _get_spiral_cache_empty()

@register_cell_magic
def spiral(arg, cell, test=False):
    global _spiral_cache
    global _notebook_name

    last_spiral_cache = _spiral_cache

    args = arg.split(' ')
    def get_arg(i, d=''): return next(iter(args[i:i+1]), d)
    arg = args[0]

    def split_imports(code): return util.list_partition(code.splitlines(), lambda line: line.startswith('open '))
    cache_imports, cache_exports = split_imports(_spiral_cache[arg])
    
    def cwpath(*arg): return os.path.abspath(os.path.join(os.getcwd(), '..', *arg))

    inplace = cell.strip() == '' and arg == ''
    if inplace:
        cell = util.read_file(cwpath('main.spi'))

    cell_imports, cell_exports = split_imports(cell)
    cell_exports = '\n'.join(cell_exports).strip('\n').split('\n')

    if arg == 'run':
        cell_imports = ['open lib_spi.util', 'open lib_spi.console_fsx'] + cell_imports
        cell_exports = ['inl main () : () = '] + list(map(lambda l: f'    {l}', cell_exports))

    spi_imports = cache_imports + cell_imports
    spi_exports = cache_exports + ([] if cell_exports == [''] else (['', '// cell'] + cell_exports))

    def join_imports(imports, exports): 
        return ''.join([
            '\n'.join(list(dict.fromkeys(imports))),
            '' if len(imports) == 0 or len(exports) == 0 else '\n\n\n',
            '\n'.join(exports).strip('\n') + '\n'
        ])
    new_code_spi = join_imports(spi_imports, spi_exports)
    
    if _notebook_name is None:
        _notebook_name = util.get_notebook_name()

    if arg in ['spi', 'spir']:
        spi_path = cwpath('lib_spi', f'{_notebook_name}.{arg}')
    else:
        spi_path = cwpath('main.spi')
    
    _spiral_cache[arg] = new_code_spi

    if test:
        return new_code_spi
    else:
        old_code_spi = util.read_file(spi_path)
        if new_code_spi != old_code_spi:
            if arg in ['run', '']: 
                if not inplace:
                    shutil.copyfile(cwpath('main.spi'), cwpath('main.spi.tmp'))
                shutil.copyfile(cwpath('main.fsx'), cwpath('main.fsx.tmp'))
                
            try:
                if not inplace:
                    util.write_file(spi_path, new_code_spi)

                if arg in ['run', '']:
                    fsx_path = cwpath("lib_fsx", "_ipython_spi.fsx" if arg == 'run' else f'{_notebook_name}_spi.fsx')
                    util.write_file(fsx_path, '')
                    
                    run_node_output = util.run_node(
                        '\n'.join([
                            f'import * as spiral_api from "../lib_ts/spiral_api"',
                            f'await spiral_api.spiToFsx("", "{fsx_path}")'
                        ]), 
                        timeout=int(get_arg(1, 10))
                    ).splitlines()

                    new_code_fsx = ''
                    start = time.time()
                    while new_code_fsx == '' and time.time() - start < 10:
                        time.sleep(0.2)
                        new_code_fsx = util.read_file(fsx_path).strip(" \n")
                    
                    if arg == 'run':
                        util.write_file(fsx_path, '')

                    print({
                        'run_node_output': run_node_output,
                        'fsx_path': fsx_path,
                        'len(new_code_fsx)': len(new_code_fsx),
                        'new_code_fsx[:100]': new_code_fsx[:100],
                    })

            except Exception as e:
                print(f'ipython_magic.spiral() error. new_code_spi={new_code_spi}')
                _spiral_cache = last_spiral_cache
                raise e
            finally:
                if arg in ['run', '']: 
                    if not inplace:
                        os.rename(cwpath('main.spi.tmp'), cwpath('main.spi'))
                    os.rename(cwpath('main.fsx.tmp'), cwpath('main.fsx'))

        return spi_path

@register_cell_magic
def spi(arg, cell, test=False):
    return spiral('spi', cell, test)

@register_cell_magic
def spir(arg, cell, test=False):
    return spiral('spir', cell, test)


# spiral test

In [5]:
_spiral_cache = _get_spiral_cache_empty()

def test_cell_spiral(arg, cell, expected_spi):
    new_code_spi = spiral(arg, cell, test=True)

    if new_code_spi != expected_spi:
        raise Exception('\n' + '\n'.join(['\n|||arg:', arg, '\n|||cell:', cell, 
                        '\n|||new_code_spi:', new_code_spi, '\n|||expected_spi:', expected_spi]))
    else:
        print('arg:', arg)

test_cell_spiral(
    'spi',
    'open a',
    'open a\n'
)

test_cell_spiral(
    'spi',
    'open a\ninl get1 () = 1i32',
    'open a\n\n\n// cell\ninl get1 () = 1i32\n'
)

test_cell_spiral(
    'spi',
    'open b\ninl get2 () = 2i32',
    'open a\nopen b\n\n\n// cell\ninl get1 () = 1i32\n\n// cell\ninl get2 () = 2i32\n'
)

test_cell_spiral(
    'run',
    'open b\ninspect 1i32',
    'open lib_spi.util\nopen lib_spi.console_fsx\nopen b\n\n\n// cell\ninl main () : () = \n    inspect 1i32\n'
)

test_cell_spiral(
    'spi',
    'open b\ninl get3 () = 3i32',
    'open a\nopen b\n\n\n// cell\ninl get1 () = 1i32\n\n// cell\ninl get2 () = 2i32\n\n// cell\ninl get3 () = 3i32\n'
)

_spiral_cache = _get_spiral_cache_empty()

<magic:201ea77668e84f79ba139ece064dfc79>


<IPython.core.display.Javascript object>

arg: spi
arg: spi
arg: spi
arg: run
arg: spi


# spiral run

In [6]:
%%node run

import * as spiral_api from "./spiral_api"
import * as path from "path"

await Promise.all(
    [
        undefined, 
        path.join(process.cwd(), '..', 'main.spi')
    ].map(async (spi) => {
        const fsx = await spiral_api.spiToFsx(spi)
        console.log({ spi, fsx })
    })
)

['{',
 '  spi: undefined,',
 '  fsx: \'#r "nuget: Feliz.JSX.Solid, 1.0.0-beta-001"\\n\' +',
 "    'open System\\n' +",
 "    'open Fable.Core\\n' +",
 "    'open Fable.Core.JsInterop\\n' +",
 "    'open Feliz.JSX.Solid\\n' +",
 "    '\\n' +",
 "    'type Mut0 = {mutable l0 : obj}\\n' +",
 "    'and [<Struct>] US0 =\\n' +",
 "    '    | US0_0\\n' +",
 "    '    | US0_1 of f1_0 : int32\\n' +",
 "    'and Heap0 = {l0 : US0}\\n' +",
 "    'and [<Struct>] US1 =\\n' +",
 "    '    | US1_0\\n' +",
 "    '    | US1_1 of f1_0 : bool\\n' +",
 "    'and [<Struct>] US2 =\\n' +",
 "    '    | US2_0\\n' +",
 "    '    | US2_1 of f1_0 : (obj -> ((obj -> unit) -> unit))\\n' +",
 "    'and [<Struct>] US3 =\\n' +",
 "    '    | US3_0\\n' +",
 "    '    | US3_1 of f1_0 : string\\n' +",
 "    'and [<Struct>] US4 =\\n' +",
 "    '    | US4_0\\n' +",
 "    '    | US4_1 of f1_0 : JS.Function\\n' +",
 "    'and [<Struct>] US5 =\\n' +",
 "    '    | US5_0\\n' +",
 "    '    | US5_1 of f1_0 : (unit -> JSX.Eleme

In [7]:
%%spiral run

inspect "test"


{'run_node_output': [], 'fsx_path': '/workspaces/tictactoe_spiral/lib_fsx/_ipython_spi.fsx', 'len(new_code_fsx)': 45, 'new_code_fsx[:100]': 'let v0 : string = "test"\nprintfn $"%A{v0}"\n()'}


'/workspaces/tictactoe_spiral/main.spi'